# Reporte EDA

In [106]:
#IMPORTS
import pandas as pd 
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
from matplotlib import cm
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import re
import urllib
from matplotlib import image as mpimg


In [107]:
# Carga datasets
customers_dataset = pd.read_csv('dataset/olist_customers_dataset.csv')
geolocation_dataset = pd.read_csv('dataset/olist_geolocation_dataset.csv')
order_items_dataset = pd.read_csv('dataset/olist_order_items_dataset.csv')
order_payments_dataset = pd.read_csv('dataset/olist_order_payments_dataset.csv')
order_reviews_dataset = pd.read_csv('dataset/olist_order_reviews_dataset.csv')
orders_dataset = pd.read_csv('dataset/olist_orders_dataset.csv')
products_dataset = pd.read_csv('dataset/olist_products_dataset.csv')
sellers_dataset = pd.read_csv('dataset/olist_sellers_dataset.csv')

df_list = [customers_dataset, geolocation_dataset, order_items_dataset, order_payments_dataset, order_reviews_dataset, orders_dataset, products_dataset, sellers_dataset]
names_list = ['customers', 'geolocation', 'order_items', 'order_payments', 'order_reviews', 'orders', 'products', 'sellers']


In [108]:
# Entradas y columnaas
print("Customers Dataset: ", customers_dataset.shape)
print("Geolocation Dataset: ", geolocation_dataset.shape)
print("Order Items Dataset: ", order_items_dataset.shape)
print("Order Payments Dataset: ", order_payments_dataset.shape)
print("Order Reviews Dataset: ", order_reviews_dataset.shape)
print("Orders Dataset: ", orders_dataset.shape)
print("Products Dataset: ", products_dataset.shape)
print("Sellers Dataset: ", sellers_dataset.shape)

Customers Dataset:  (99441, 5)
Geolocation Dataset:  (1000163, 5)
Order Items Dataset:  (112650, 7)
Order Payments Dataset:  (103886, 5)
Order Reviews Dataset:  (99224, 7)
Orders Dataset:  (99441, 8)
Products Dataset:  (32951, 9)
Sellers Dataset:  (3095, 4)


## Evaluación de calidad de datos

### Columnas Duplicadas y Valores Faltantes

Para evaluar la calidad de los datos y realizar la limpieza, primero vamos a definir una funcion de utilidad para resumir brevemente cada uno de los DF's con los que estaremos trabajando

In [109]:
def summarize_dataframes(df_list, names_list):

    all_summaries = []
    
    # Para cada DF
    for df, name in zip(df_list, names_list):
        # Para cada columna del DF
        for col in df.columns:
            # Cantidad / porcentaje de valores nulos
            null_qty = df[col].isnull().sum()
            null_pct = null_qty / len(df) * 100

            # Tipo
            dtype = df[col].dtype

            # Cantidad de Entradas Unicas
            cat_qty = df[col].nunique()

            # Agregar informacion al summary
            all_summaries.append({
                'dataset': name,
                'feature': col,
                'null_qty': null_qty,
                'null_pct': null_pct,
                'type': dtype,
                'cat_qty': cat_qty
            })
    
    summary_df = pd.DataFrame(all_summaries)
    return summary_df

summarize_dataframes(df_list, names_list)

,dataset,feature,null_qty,null_pct,type,cat_qty
0,customers,customer_id,0,0.000000,object,99441
1,customers,customer_unique_id,0,0.000000,object,96096
2,customers,customer_zip_code_prefix,0,0.000000,int64,14994
3,customers,customer_city,0,0.000000,object,4119
4,customers,customer_state,0,0.000000,object,27
5,geolocation,geolocation_zip_code_prefix,0,0.000000,int64,19015
6,geolocation,geolocation_lat,0,0.000000,float64,717360
7,geolocation,geolocation_lng,0,0.000000,float64,717613
8,geolocation,geolocation_city,0,0.000000,object,8011
9,geolocation,geolocation_state,0,0.000000,object,27


A primera vista, tenemos algunas columnas con una alta cantidad de valores nulos. Sin embargo, entre ellas encontramos algunas como 'Review Comment Title' o 'Review Comment Message' donde tiene sentido que tengan una alta cantidad de valores nulos y estos nos proveen informacion. En este caso, vamos a transformar las variables a 'has_title' y 'has_comment' para poder utilizar esta informacion. En los demas casos, podemos simplemente eliminar los valores nulos al ser una parte poco significativa del dataset.

In [110]:
# Eliminamos columnas con valores nulos para las ordenes
orders_dataset = orders_dataset.dropna(subset=['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date'])

# Eliminamos columnas con valores nulos para productos
products_dataset = products_dataset.dropna(subset=['product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_length_cm',
                                                   'product_height_cm', 'product_width_cm'])

# Transformacion de variables
order_reviews_dataset['has_title'] = order_reviews_dataset['review_comment_title'].notna()
order_reviews_dataset['has_comment'] = order_reviews_dataset['review_comment_message'].notna()

order_reviews_dataset = order_reviews_dataset.drop(columns=['review_comment_title', 'review_comment_message'])

df_list = [customers_dataset, geolocation_dataset, order_items_dataset, order_payments_dataset, order_reviews_dataset, orders_dataset, products_dataset, sellers_dataset]

summarize_dataframes(df_list, names_list)

,dataset,feature,null_qty,null_pct,type,cat_qty
0,customers,customer_id,0,0.0,object,99441
1,customers,customer_unique_id,0,0.0,object,96096
2,customers,customer_zip_code_prefix,0,0.0,int64,14994
3,customers,customer_city,0,0.0,object,4119
4,customers,customer_state,0,0.0,object,27
5,geolocation,geolocation_zip_code_prefix,0,0.0,int64,19015
6,geolocation,geolocation_lat,0,0.0,float64,717360
7,geolocation,geolocation_lng,0,0.0,float64,717613
8,geolocation,geolocation_city,0,0.0,object,8011
9,geolocation,geolocation_state,0,0.0,object,27


Luego de limpiar los valores nulos, vamos a eliminar filas duplicadas en cada uno de los dataframes

In [ ]:
print("Customers Dataset: ", customers_dataset.shape)
print("Geolocation Dataset: ", geolocation_dataset.shape)
print("Order Items Dataset: ", order_items_dataset.shape)
print("Order Payments Dataset: ", order_payments_dataset.shape)
print("Order Reviews Dataset: ", order_reviews_dataset.shape)
print("Orders Dataset: ", orders_dataset.shape)
print("Products Dataset: ", products_dataset.shape)
print("Sellers Dataset: ", sellers_dataset.shape)

Customers Dataset:  (99441, 5)
Geolocation Dataset:  (1000163, 5)
Order Items Dataset:  (112650, 7)
Order Payments Dataset:  (103886, 5)
Order Reviews Dataset:  (99224, 7)
Orders Dataset:  (96461, 8)
Products Dataset:  (32340, 9)
Sellers Dataset:  (3095, 4)
Customers Dataset:  (99441, 5)
Geolocation Dataset:  (738332, 5)
Order Items Dataset:  (112650, 7)
Order Payments Dataset:  (103886, 5)
Order Reviews Dataset:  (99224, 7)
Orders Dataset:  (96461, 8)
Products Dataset:  (32340, 9)
Sellers Dataset:  (3095, 4)


### Customers

In [117]:
customers_dataset.head(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,SP
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG


El dataframe de customers unicamente contiene informacion pertinente al proyecto, al haberlo procesado en conjunto a los demas en el inciso anterior no debemos realizar modificaciones adicionales.

### Geolocations

In [118]:
geolocation_dataset.head(10)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
5,1012,-23.547762,-46.635361,são paulo,SP
6,1047,-23.546273,-46.641225,sao paulo,SP
7,1013,-23.546923,-46.634264,sao paulo,SP
8,1029,-23.543769,-46.634278,sao paulo,SP
9,1011,-23.547640,-46.636032,sao paulo,SP


### Geolocation


### Resúmenes estadísticos

In [113]:
###

### Visualización de patrones clave

In [114]:
###

### Análisis de correlación de características

In [115]:
###